# Using Ollama to get structured output

To structure text, using a structured output enables better extraction and validation.

First setup the notebook.

In [31]:
#load "load.fsx"
    
open System

open Newtonsoft.Json
open Informedica.Utils.Lib.BCL

open Informedica.OpenAI.Lib
open Ollama.Operators

let extraction = function
| Ok x -> printfn $"## Extracted:\n{x}"
| Error err -> printfn $"## Extraction failed:\n{err}"

let messages =
    [
        """
You are a medical and pharmaceutical expert specifically quallified to answer questions
about medication and dosing medication. You also know about measurements and units.
You are asked to extract data from free text and structure the data in JSON.
"""
        |> Message.system
    ]

## First the hello world of LLMs to check everthing is running

In [32]:
"You are a helpful assistant"
|> init Ollama.Models.llama2
>>? "Why is the sky blue?"
|> Conversation.print

Starting conversation with llama2

Options:
{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penalty":null,"mirostat":0,"mirostat_tau":null,"mirostat_eta":null,"penalize_newline":null,"stop":[],"numa":null,"num_ctx":2048,"num_batch":null,"num_gqa":null,"num_gpu":null,"main_gpu":null,"low_vram":null,"f16_kv":null,"vocab_only":null,"use_mmap":null,"use_mlock":null,"rope_frequency_base":null,"rope_frequency_scale":null,"num_thread":null}

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"messages":[{"content":"You are a helpful assistant","role":"system"},{"content":"Why is the sky blue?","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penalty

## Define a schema and a type for the output

The json function will output the type used as a type parameter. However, due to limitations of the ollama framework you need to add the schema to the prompt as well.

In [33]:

$"""
Use schema: { "{| number: int; unit: string |}" |> Utils.anonymousTypeStringToJson}
What is the minimal corrected gestational age mentioned in the text between '''

'''A neonate 28 weeks to 32 weeks corrected gestational age.'''

Reply in JSON."""
|> Message.user
|> Ollama.json<{| number: int; unit: string |}>
    Ollama.Models.llama2
    messages
|> Async.RunSynchronously
|> extraction

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema: { number: int unit: string }\nWhat is the minimal corrected gestational age mentioned in the text between '''\n\n'''A neonate 28 weeks to 32 weeks corrected gestational age.'''\n\nReply in JSON.","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penalty":null,"mirostat":0,"mirostat_tau":null,"mirostat_eta":null,"penalize_newline":null,"stop":[],"numa":null,"num_ctx":2048,"num_batch":null,"num_gqa":null,"num

## Getting  the maximum age as a json structure

The above output is correct, now try and get the maximum age.

In [34]:
$"""
Use schema: { "{| number: int; unit: string |}" |> Utils.anonymousTypeStringToJson }
What is the maximum corrected gestational age mentioned in the text between '''

'''A neonate 28 weeks to 32 weeks corrected gestational age.'''

Reply in JSON."""
|> Message.user
|> Ollama.json<{| number: int; unit: string |}>
    Ollama.Models.llama2
    messages
|> Async.RunSynchronously
|> extraction

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema: { number: int unit: string }\nWhat is the maximum corrected gestational age mentioned in the text between '''\n\n'''A neonate 28 weeks to 32 weeks corrected gestational age.'''\n\nReply in JSON.","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penalty":null,"mirostat":0,"mirostat_tau":null,"mirostat_eta":null,"penalize_newline":null,"stop":[],"numa":null,"num_ctx":2048,"num_batch":null,"num_gqa":null,"num

## Try a different structured output

Somehow, the prompt is misunderstood and the minimum age value is returned instead of the maximum age value.

Let's try again using a more complex structure.

In [35]:
$"""
Use schema: { "{| number: int; unit: string |}" |> Utils.anonymousTypeStringToJson }
What is corrected gestational age range mentioned in the text between '''

'''A neonate 28 weeks to 32 weeks corrected gestational age.'''

Reply in JSON."""
|> Message.user
|> Ollama.json<{| minAge: int; maxAge: int; unit: string |}>
    Ollama.Models.llama2
    messages
|> Async.RunSynchronously
|> extraction

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema: { number: int unit: string }\nWhat is corrected gestational age range mentioned in the text between '''\n\n'''A neonate 28 weeks to 32 weeks corrected gestational age.'''\n\nReply in JSON.","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penalty":null,"mirostat":0,"mirostat_tau":null,"mirostat_eta":null,"penalize_newline":null,"stop":[],"numa":null,"num_ctx":2048,"num_batch":null,"num_gqa":null,"num_gpu":

## Extraction with different units

Now try a Dutch text with differrent units for the minimum and maximum age.

In [36]:
$"""
Use schema: { "{| minAge: int; maxAge: int; minAgeUnit: string; maxAgeUnit: string |}" |> Utils.anonymousTypeStringToJson }
What is age range mentioned in the text between '''

'''
paracetamol
Oraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.
'''

Respond in JSON
"""
|> Message.user
|> Ollama.json<{| minAge: int; maxAge: int; minAgeUnit: string; maxAgeUnit: string |}>
    Ollama.Models.llama2
    messages
|> Async.RunSynchronously
|> extraction

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema: { minAge: int maxAge: int minAgeUnit: string maxAgeUnit: string }\nWhat is age range mentioned in the text between '''\n\n'''\nparacetamol\nOraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.\n'''\n\nRespond in JSON\n","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_

## Use a more explicit structure

A more explicit structure also has more semantic meaning. The below structure is an explicit range structure with a min and a max object containing an age structure.

In [37]:

$"""
Use schema: { "{| ageRange : {| minAge: {| age: int; unit: string |}; maxAge: {| age: int; unit: string |} |} |}" |> Utils.anonymousTypeStringToJson }
What is age range mentioned in the text between '''

'''
paracetamol
Oraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.
'''

Respond in JSON
"""
|> Message.user
|> Ollama.json<{| ageRange : {| minAge: {| age: int; unit: string |}; maxAge: {| age: int; unit: string |} |} |} >
    Ollama.Models.llama2
    messages
|> Async.RunSynchronously
|> extraction

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema: { ageRange : { minAge: { age: int unit: string } maxAge: { age: int unit: string } } }\nWhat is age range mentioned in the text between '''\n\n'''\nparacetamol\nOraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.\n'''\n\nRespond in JSON\n","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_pen

## A more demanding extraction with different units

Try to extract 6 months - 1 year. So, naively 6 > 1 but with units ofcourse not so!

In [38]:
$"""
Use schema: { "{| ageRange : {| minAge: {| age: int; unit: string |}; maxAge: {| age: int; unit: string |} |} |}" |> Utils.anonymousTypeStringToJson }
What is age range mentioned in the text between '''

'''
paracetamol
Oraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 6 maanden – 1 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.
'''

Respond in JSON
"""
|> Message.user
|> Ollama.json<{| ageRange : {| minAge: {| age: int; unit: string |}; maxAge: {| age: int; unit: string |} |} |} >
    Ollama.Models.llama2
    messages
|> Async.RunSynchronously
|> extraction

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema: { ageRange : { minAge: { age: int unit: string } maxAge: { age: int unit: string } } }\nWhat is age range mentioned in the text between '''\n\n'''\nparacetamol\nOraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 6 maanden – 1 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.\n'''\n\nRespond in JSON\n","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_

Surprise! It figured out that 1 year = 12 months, so the max age is indeed 12 months, i.e. 1 year. At first glance I actually thought the LLM got it wrong ;-)

## Try the same with Fireworkds API

In [39]:
let msg =
    $"""
Use schema: { "{| ageRange : {| minAge: {| age: int; unit: string |}; maxAge: {| age: int; unit: string |} |} |}" |> Utils.anonymousTypeStringToJson }
What is age range mentioned in the text between '''

'''
paracetamol
Oraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 6 maanden – 1 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.
'''

Respond in JSON
"""
    |> Message.user

Fireworks.Chat.defaultChatInput
    "accounts/fireworks/models/llama-v2-70b-chat"
    msg messages
|> Fireworks.chatJson<{| ageRange : {| minAge: {| age: int; unit: string |}; maxAge: {| age: int; unit: string |} |} |} >
|> Async.RunSynchronously
|> extraction

ℹ INFO: 
EndPoint: https://api.fireworks.ai/inference/v1/chat/completions
Payload:
{"model":"accounts/fireworks/models/llama-v2-70b-chat","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema: { ageRange : { minAge: { age: int unit: string } maxAge: { age: int unit: string } } }\nWhat is age range mentioned in the text between '''\n\n'''\nparacetamol\nOraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 6 maanden – 1 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.\n'''\n\nRespond in JSON\n","role":"user"}],"tools":[],"max_tokens":200,"prompt_truncate_len":1500,"temperature":0.0,"top_p":1.0,"top_k":50,"frequency_penalty

## Extract a dose structure

Let's try to extract a dose from a text.

In [40]:
"""
Use schema: { maxDose: float, unit: string }
What is the max dose mentioned in the text between '''

'''
paracetamol
Oraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.
'''

Respond in JSON
"""
|> Message.user
|> Ollama.json<{| maxDose: int; unit: string |}>
    Ollama.Models.llama2
    messages
|> Async.RunSynchronously
|> extraction

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema: { maxDose: float, unit: string }\nWhat is the max dose mentioned in the text between '''\n\n'''\nparacetamol\nOraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.\n'''\n\nRespond in JSON\n","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penal

## Validate an extraction

We wanted to get the absolute maximum dose. We can add a validator making sure the unit is just the dose unit and a time unit.

In [41]:
let validator =
    let isTimeUnit s =
        ["dag"; "day"; "uur"; "hour"; "week"]
        |> List.exists (fun tu -> s |> String.contains tu)
    fun s ->
        let vu = s |> JsonConvert.DeserializeObject<{| maxDose: float; doseUnit: string; timeUnit : string |}>
        if vu.timeUnit |> isTimeUnit then 
            if vu.doseUnit |> String.split "/" |> List.length = 1 then s |> Ok 
            else
                $"{vu.doseUnit} should be a single dose unit" 
                |> Error
        else 
            $"{vu.timeUnit} is not a time unit but should be a time unit"
            |> Error

In [42]:
$"""
Use schema: { "{| maxDose: float; doseUnit: string; timeUnit : string |}" |> Utils.anonymousTypeStringToJson }
What is the max dose mentioned in the text between '''

'''
paracetamol
Oraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.
'''

Respond in JSON
"""
|> Message.user
|> fun msg -> 
    { msg with
        Validator = validator
    }
|> Ollama.validate<{| maxDose: float; doseUnit: string; timeUnit : string |}>
    Ollama.Models.llama2
    messages
|> Async.RunSynchronously
|> extraction

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"format":"json","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema: { maxDose: float doseUnit: string timeUnit : string }\nWhat is the max dose mentioned in the text between '''\n\n'''\nparacetamol\nOraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.\n'''\n\nRespond in JSON\n","role":"user"}],"model":"llama2","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":

In [43]:
let msg = 
    $"""
Use schema { "{| maxDose: float; doseUnit: string; timeUnit : string |}" |> Utils.anonymousTypeStringToJson }
What is the max dose mentioned in the text between '''

'''
paracetamol
Oraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.
'''

Respond in JSON
"""
    |> Message.user

Fireworks.Chat.defaultChatInput
    "accounts/fireworks/models/llama-v2-70b-chat"
    msg messages
|> Fireworks.chatJson<{| maxDose: float; doseUnit: string; timeUnit : string |}>
|> Async.RunSynchronously
|> Result.map (fun resp ->
    resp.Response
    |> _.choices
    |> List.last
    |> _.message.content
    |> JsonConvert.DeserializeObject<{| maxDose: float; unit: string |}>
)
|> extraction

ℹ INFO: 
EndPoint: https://api.fireworks.ai/inference/v1/chat/completions
Payload:
{"model":"accounts/fireworks/models/llama-v2-70b-chat","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema { maxDose: float doseUnit: string timeUnit : string }\nWhat is the max dose mentioned in the text between '''\n\n'''\nparacetamol\nOraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.\n'''\n\nRespond in JSON\n","role":"user"}],"tools":[],"max_tokens":200,"prompt_truncate_len":1500,"temperature":0.0,"top_p":1.0,"top_k":50,"frequency_penalty":0.0,"presence_penalty":0.0,"n":1

In [44]:
let msg = 
    $"""
Use schema { "{| maxDose: float; doseUnit: string; timeUnit : string |}" |> Utils.anonymousTypeStringToJson }
What is the max dose mentioned in the text between '''

'''
paracetamol
Oraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.
'''

Respond in JSON
"""
    |> Message.user

Fireworks.Chat.defaultChatInput
    "accounts/fireworks/models/llama-v2-70b-chat"
    msg messages
|> Fireworks.validate<{| maxDose: float; doseUnit: string; timeUnit : string |}>
    validator
|> Async.RunSynchronously
|> extraction

ℹ INFO: 
EndPoint: https://api.fireworks.ai/inference/v1/chat/completions
Payload:
{"model":"accounts/fireworks/models/llama-v2-70b-chat","messages":[{"content":"\nYou are a medical and pharmaceutical expert specifically quallified to answer questions\nabout medication and dosing medication. You also know about measurements and units.\nYou are asked to extract data from free text and structure the data in JSON.\n","role":"system"},{"content":"\nUse schema { maxDose: float doseUnit: string timeUnit : string }\nWhat is the max dose mentioned in the text between '''\n\n'''\nparacetamol\nOraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.\n'''\n\nRespond in JSON\n","role":"user"}],"tools":[],"max_tokens":200,"prompt_truncate_len":1500,"temperature":0.0,"top_p":1.0,"top_k":50,"frequency_penalty":0.0,"presence_penalty":0.0,"n":1